# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS album_library "
query = query + "(artist text, song_title text, song_length double, session_id int, itemInSession int, PRIMARY KEY(session_id, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [15]:
file = 'event_datafile_new.csv'

insertQuery = "INSERT INTO album_library (artist, song_title, song_length, session_id, itemInSession)"
insertQuery = insertQuery + " VALUES (?, ?, ?, ?, ?)"
#artist 0, firstName 1, gender 2, itemSession 3, lastName 4, length 5, level 6, location 7, sessionId 8, song 9, userId 10
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        prepared = session.prepare(insertQuery)
        session.execute(prepared, (line[0], line[9], line[5], line[8], line[3]))

TypeError: Received an argument of invalid type for column "song_length". Expected: <class 'cassandra.cqltypes.DoubleType'>, Got: <class 'str'>; (required argument is not a float)

In [27]:
df = pd.read_csv("event_datafile_new.csv")

In [28]:
try:
    for index, r in df.iterrows():
        session.execute(insertQuery, (r.artist, r.song, r.length, r.sessionId, r.itemInSession))
except Exception as e:
    print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [33]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute("select artist, song_title, song_length from album_library where session_id = 338 and itemInSession = 4")
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.3073)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [35]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query = query + "(sessionId int, userId int, itemInSession int, artist_name text, song text, firstName text, lastName text, PRIMARY KEY((sessionId, userId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                  

In [40]:
insertQuery = "INSERT INTO song_playlist_session (sessionId, userId, itemInSession, artist_name, song, firstName, lastName)"
insertQuery = insertQuery + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

In [41]:
try:
    for index, r in df.iterrows():
        session.execute(insertQuery, (r.sessionId, r.userId, r.itemInSession, r.artist, r.song, r.firstName, r.lastName))
except Exception as e:
    print(e)

In [42]:
try:
    rows = session.execute("select artist_name, song, firstName, lastName from song_playlist_session where sessionId = 182 and userId = 10")
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist_name='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist_name='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist_name='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist_name='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


In [45]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = "CREATE TABLE IF NOT EXISTS user_songs "
query = query + "(song text, userId int, artist_name text, firstName text, lastName text, sessionId int, itemInSession int, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)                  

In [46]:
insertQuery = "INSERT INTO user_songs (song, userId, artist_name, firstName, lastName, sessionId, itemInSession)"
insertQuery = insertQuery + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

In [48]:
try:
    for index, r in df.iterrows():
        session.execute(insertQuery, (r.song, r.userId, r.artist, r.firstName, r.lastName, r.sessionId, r.itemInSession))
except Exception as e:
    print(e)

In [49]:
try:
    rows = session.execute("select firstName, lastName from user_songs where song = 'All Hands Against His Own'")
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [54]:
queryDropLibrary = "drop table album_library"
queryDropUserSongs = "drop table user_songs3"
try:
    session.execute(queryDropLibrary)
    session.execute(queryDropUserSongs)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [55]:
session.shutdown()
cluster.shutdown()